### 3.2 정기적으로 실행하기
- schedule_interval를 입력안하면 None이랑 비슷하다. 수동으로 실행해야됨
- @Daily로 설정한 경우 매일 자정에 실행 됨
- 종료일이 없으면 DAG를 종료할 때까지 실행됩니다.

In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="01_unscheduled", 
    start_date=datetime(2019, 1, 1), # DAG 실행 스케줄을 시작할 날짜/사간
    schedule_interval="@Daily"
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json http://events_api:5000/events"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    Path(output_path).parent.mkdir(exist_ok=True)

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats

### 3.2.2 Cron 기반의 스케줄 간격 설정하기
- *(분) *(시) *(일) *(월) *(요일)
- 0 * * * * = 매시간
- 0 0 * * * = 매일
- 0 0 * * 0 = 매주
- 0 0 1 * * = 매월 1일 자정
- 45 23 * * SAT = 매주 토요일 23시 45분
- @once : 1회만 실행하도록 스케줄
- @hourly : 매시간 변경시 1회 실행
- @daily : 매일 자정 1회
- @weekly : 매주 일요일 자정 1회
- @monthly : 매월 1일 자정 1회
- @yearly : 매년 1월 1일 자정에 1회

### 3.2.3 빈도 기반의 스케줄 간격 설정하기
- 예륻 들어 3일에 한 번씩 실행 : schedule_interval = datetime.timedelta(days=3)

### 3.3.2 실행날짜를 사용하여 동적 시간 참조하기
- DAG가 실행되는 날짜와 시간을 나타내는 매개변수는 excution_date 입니다.
- 다음 실행일 next_excution_date입니다.
- 과거의 스케줄 간격의 시작은 previous_excution_date
- 축약 매개변수를 제공합니다. ds 및 ds_nodash는 각각 YYYY-MM-DD, YYYYMMDD 형식으로 된 excution_date
- next_ds, next_ds_nodas, prev_ds, prev_ds_nodash

In [ ]:
"""
fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json "
        "http://events_api:5000/events?"
        "start_date={{execution_date.strftime('%Y-%m-%d')}}&"
        "end_date={{next_execution_date.strftime('%Y-%m-%d')}}"
    ),
    dag=dag,
)
"""
"""
fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json "
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)
"""